# Notebook: Bài toán Answer-Extraction
Bài toán "Answer Extraction" hay "Extractive Question Answering" là một phần quan trọng của hệ thống trả lời câu hỏi (QA). Trong bài toán này, mục tiêu là trích xuất một phần của văn bản từ tài liệu nguồn mà trả lời chính xác cho câu hỏi của người dùng. Thông thường, bài toán Answer Extraction đòi hỏi khả năng xác định và phân loại các đoạn văn bản có để tìm ra câu trả lời hoặc các phần của câu trả lời.

![image](https://qa.fastforwardlabs.com/images/copied_from_nb/my_icons/QAworkflow.png)

Trong notebook này, chúng ta sẽ cùng giải quyết các bài toán liên quan đến Answer Extraction. Chúng ta có thể giải quyết bài toán này bằng cách giải quyết 1 trong 2 biến thể sau:
1. Tìm điểm bắt đầu và điểm kết thúc của đoạn nội dung có liên quan tới câu hỏi.
2. Bài toán phân loại câu: tìm những câu trong văn bản có nội dung liên quan tới câu hỏi

In [ ]:
%%capture
# @title Cài đặt thư viện
model_checkpoint = "distilbert-base-uncased" # @param {type:"string"}
max_length = 384 # @param {type:"string"} # Độ dài tối đa của input đầu vào
doc_stride = 128 # @param {type:"string"} # Đoạn trùng nhau giữa 2 phần nếu như cắt

!pip install datasets transformers accelerate git-lfs

# Đăng ký tài khoản huggingface và tạo token để đăng nhập phục vụ việc upload và download mô hình và dữ liệu
# from huggingface_hub import notebook_login
# notebook_login()

# Kiểm tra việc cài đặt đã hoàn thành và phiên bản
import transformers
from transformers import AutoTokenizer
transformers.__version__

## Giải quyết bài toán Answer-Extraction theo hướng tìm vị trí (bắt đầu - kết thúc)
---
Mục tiêu của bài toán này sẽ là tìm vị trí bắt đầu và kết thúc của đoạn nội dung liên quan tới câu hỏi. Notebook sẽ hướng dẫn finetune một mô hình ngôn ngữ đã được huấn luyện trước (pretrained model) [🤗 Transformers](https://github.com/huggingface/transformers) cho bài toán trả lời câu hỏi bằng phương pháp trích xuất (extractive question answering). Bài toán có thể được mô tả như sau:

**Input:**
  - Question (câu hỏi)
  - Context (Đoạn văn bản cần tìm ra đoạn trả lời)

**Output:**
  - (start_index, end_index) vị trí bắt đầu và kết thúc của đoạn văn bản trong context.

**Note:** Mô hình chỉ sử dụng thông tin trong ngữ cảnh để trả lời, không sinh ra câu trả lời.

---

In [ ]:
# @title Load bộ dữ liệu SQuAd
# V2 là bản cải tiến của V1 được thêm vào những câu hỏi không có câu trả lời
squad_v2 = False # @param {type:"boolean"}


from datasets import load_dataset, load_metric
from IPython.display import clear_output
from pprint import pprint
datasets = load_dataset("squad_v2" if squad_v2 else "squad")
clear_output()
print('Overview datasets: ', datasets)
print('-'*50)
print('Overview a sample: ')
pprint(datasets["train"][0])

Overview datasets:  DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})
--------------------------------------------------
Overview a sample: 
{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grott

### Giải thích tên các trường:
---
- id: chỉ số
- title: tiêu đề
- context: ngữ cảnh của câu hỏi
- question: câu hỏi
- answers:
  * answer_start: là một mảng chứa các vị trí bắt đầu của câu trả lời. (có thể rỗng)
  * text: là một mảng chứa các câu trả lời tương ứng với từng vị trí ở mảng answer_start. (có thể rỗng)
---



### Tiền xử lý dữ liệu huấn luyện
---
Trước khi đưa dữ liệu vào mô hình ta cần tiền xử lý chúng.

1. Ở bước tiền xử lý đầu tiên, ta sẽ sử dụng một 🤗 Transformers `Tokenizer` để tokenize dữ liệu đầu vào. Ta sử dụng phương thức `AutoTokenizer.from_pretrained` để tạo ra một đối tượng `Tokenizer`.

  **Note:** Tokenizer có thể cần giống với mô hình sử dụng nhưng trong đa số trường hợp nên sử dụng tokenizer đi kèm với mô hình để đạt hiệu quả tốt nhất.
2. Do vị trí của câu trả lời trước và sau khi tokenize có thể sẽ khác nhau. Chính vì vậy, bước tiền xử lý thứ hai chính là tìm ra vị trí bắt đầu và kết thúc của câu trả lời trên dữ liệu đã được tokenize.
---

### Tiền xử lý 1: Tokenize câu truy vấn và đoạn văn bản

In [ ]:
from transformers import AutoTokenizer
from pprint import pprint
from IPython.display import clear_output

# Khởi tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
clear_output()

# Thử nghiệm tokenizer
pprint(tokenizer("What is your name?", "My name is Sylvain."))
# hoặc tokenizer.encode("What is your name?", "My name is Sylvain.")

# Decode ngược lại từ input_ids về đoạn văn bản
print("Decode input-ids")
print(tokenizer.decode([101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102]))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101,
               2054,
               2003,
               2115,
               2171,
               1029,
               102,
               2026,
               2171,
               2003,
               25353,
               22144,
               2378,
               1012,
               102]}
Decode input-ids
[CLS] what is your name? [SEP] my name is sylvain. [SEP]


Kiểm tra việc sử dụng `max_length` và `truncation`.
* `max_lenght` để chỉ ra độ dài tối đa của sample sau khi đã tokenize.
* `truncation="only_second"` để chỉ cắt phần sau (context).

In [ ]:
# Tìm ra sample có độ dài sau khi tokenize lớn hơn 384
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

# Kiểm tra length sau khi tokenize của sample nếu không dùng max_length và truncation để cắt
print('Uncut: ', len(tokenizer(example["question"], example["context"])["input_ids"]))

# Kiểm tra length sau khi tokenize của sample nếu dugnf max_length và truncation để cắt
print('Cut: ', len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"]))

Uncut:  396
Cut:  384


Thêm vào tham số
* `return_overflowing_tokens=True` để giữ lại và xử lý cả những phần bị cắt

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

# Kiểm tra xem việc thêm có giữ lại được phần bị cắt không
print('Kết quả tokenize sau khi đã giữ lại những phần bị cắt: ', [len(x) for x in tokenized_example["input_ids"]])

# Decode ngược lại từ input_ids thành văn bản
print("Decode: ")
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

Kết quả tokenize sau khi đã giữ lại những phần bị cắt:  [384, 157]
Decode: 
[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is 

Thêm vào tham số
* `return_offset_mapping=True` để biết được vị trí của các token.

**Note:** `[CLS]` là ký tự đặc biệt nên sẽ có vị trí bắt đầu và kết thúc là (0, 0)

In [ ]:
from pprint import pprint
# thực hiện tokenize với tham số return_offset_mapping
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)

print('Tập các cặp vị trí (bắt đầu - kết thúc) của các token')
pprint(tokenized_example["offset_mapping"][0][:20])

print('-' * 20)

# Lấy ra token_id của token đầu tiên
first_token_id = tokenized_example["input_ids"][0][1]
# In ra token theo first_token_id
print('Lấy token theo token_id: ', tokenizer.convert_ids_to_tokens(first_token_id))

# Lấy ra cặp vị trí (bắt đầu - kết thúc) của token đầu tiên
offsets = tokenized_example["offset_mapping"][0][1]
# In ra token theo vị trí bắt đầu và kết thúc của token đó
print('Lấy token theo cặp vị trí: ', example["question"][offsets[0]:offsets[1]])

Tập các cặp vị trí (bắt đầu - kết thúc) của các token
[(0, 0),
 (0, 3),
 (4, 8),
 (9, 13),
 (14, 18),
 (19, 22),
 (23, 28),
 (29, 33),
 (34, 37),
 (37, 38),
 (38, 39),
 (40, 50),
 (51, 55),
 (56, 60),
 (60, 61),
 (0, 0),
 (0, 3),
 (4, 7),
 (7, 8),
 (8, 9)]
--------------------
Lấy token theo token_id:  how
Lấy token theo cặp vị trí:  How


### Tiền xử lý 2: Tìm vị trí câu trả lời trên chuỗi dữ liệu đã được tokenize

In [ ]:
# sequence_ids() sẽ giúp ta biết mỗi token thuộc về câu nào khi ta truyền nhiều câu vào `tokenizer`
# **Note:** `none` là giá trị đặc biệt khi token đó không thuộc về câu nào (thường là các token đặc biệt)
sequence_ids = tokenized_example.sequence_ids()
print('Sequence IDS: \n', sequence_ids)


# Sử dụng biến exmplae từ Tiền xử lý 1 để làm ví dụ
answers = example["answers"]
# Vị trí bắt đầu của câu trả lời
ans_start_char = answers["answer_start"][0]
# Vị trí kết thúc của câu trả lời
ans_end_char = ans_start_char + len(answers["text"][0])
print('Answer text: ', answers["text"][0])


# Tìm vị trí của token bắt đầu của ngữ cảnh
context_token_start_index = 0
while sequence_ids[context_token_start_index] != 1:
    context_token_start_index += 1

# Tìm vị trí token kết thúc của ngữ cảnh
context_token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[context_token_end_index] != 1:
    context_token_end_index -= 1

# Lấy ra danh sách các cặp vị trí (bắt đầu - kết thúc) của các token
offsets = tokenized_example["offset_mapping"][0]

# Kiểm tra xem câu trả lời có nằm trong ngữ cảnh hiện tại không (do ngữ cảnh dài có thể bị chia nhỏ)
if (offsets[context_token_start_index][0] <= ans_start_char and offsets[context_token_end_index][1] >= ans_end_char):
    # Nếu có nằm trong ngữ cảnh hiện tại,
    # thực hiện việc điều chỉnh context_token_start_index và context_token_end_index sao cho khớp với phần văn bản chứa câu trả lời
    while context_token_start_index < len(offsets) and offsets[context_token_start_index][0] <= ans_start_char:
        context_token_start_index += 1
    start_position = context_token_start_index - 1
    while offsets[context_token_end_index][1] >= ans_end_char:
        context_token_end_index -= 1
    end_position = context_token_end_index + 1
    print('Vị trí bắt đầu và kết thúc của câu trả lời trên chuỗi đã được tokenize: ', start_position, '->' , end_position)
else:
    print("The answer is not in this feature.")

Sequence IDS: 
 [None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Kiểm tra lại

In [ ]:
print('Câu trả lời dựa vào cặp vị trí đã tìm được: ', tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print('Câu trả lời gốc của dữ liệu: ', answers["text"][0])

Câu trả lời dựa vào cặp vị trí đã tìm được:  over 1, 600
Câu trả lời gốc của dữ liệu:  over 1,600


Trong trường hợp không có câu trả lời ta sẽ đặt vị trí bắt đầu và vị trí kết thúc của câu trả lời vào vị trí của token `[CLS]`

### Viết hàm kết hợp các bước tiền xử lý để thực hiện tiền xử lý dữ liệu

In [ ]:
# Ta cần padding nếu như ngữ cảnh hoặc context ngắn
pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):

    # Loại bỏ các khoảng trắng ở đầu hoặc cuối
    examples["question"] = [q.strip() for q in examples["question"]]

    # Tokenize dữ liệu
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )


    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Tạo mảng để lưu vị trí bắt đầu và kết thúc của câu trả lời
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # Các câu hỏi không có câu trả lời sẽ được đặt vào vị trí của token [CLS]
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Để biết vị trí của context và câu hỏi
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Do việc cắt câu nên một câu có thể nằm ở nhiều phần khác nhau ta lấy sample_index để tìm lại vị trí
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]


        # Nếu không có câu trả lời đưa vào vị trí cls_index
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Vị trí bắt đầu và kết thúc của câu trả lời
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Tìm vị trí token bắt đầu của context
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # Tìm vị trí token kết thúc của context
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Kiểm trả xem câu trả lời có nằm trong đoạn không
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                # Nếu không thì đưa vào vị trí cls_index như không có câu trả lời
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Tìm vị trí của đoạn chứa câu trả lời
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

Hàm sẽ làm việc với 1 hoặc nhiều example đồng thời

In [ ]:
features = prepare_train_features(datasets['train'][:5])

Sử dụng phương thức `map` để tạo ra dữ liệu được tokenize và xóa hết đi các cột của dữ liệu cũ

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets['train'] = tokenized_datasets['train'].select(range(100))
tokenized_datasets['validation'] = tokenized_datasets['validation'].select(range(100))

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

### Fine-tuning model

Tạo ra một đối tượng `AutoModelForQuestionAnswering` bằng cách gọi phương thức `from_pretrained`

In [ ]:
from transformers import AutoModelForQuestionAnswering

# Sử dụng model_checkpoint định nghĩa ở bước Tiền xử lý 1
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Cảnh báo kia thông báo rằng mô hình pretrain không phục vụ sẵn cho bài toán. Nên mô hình sẽ thêm và khởi tạo thêm 1 lớp ở cuối cùng của mô hình để giải quyết bài toán.

[`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) là một lớp quan trọng giúp ta chọn các siêu tham số cho mô hình

In [ ]:
from transformers import TrainingArguments
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    # push_to_hub=True,
)

Ta cần một data collator để batch dữ liệu lại trong quá trình huấn luyện trong trường hợp này ta chỉ cần sử dụng mặc định

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

Tạo ra đối tượng `Trainer` để huấn luyện mô hình

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Huấn luyện mô hình bằng phương thức train

In [ ]:
# Huấn luyện và lưu mô hình
trainer.train()
trainer.save_model("test-squad-trained")

# Lệnh để đẩy mô hình lên tài khoản cá nhân
# trainer.push_to_hub()

Epoch,Training Loss,Validation Loss
1,No log,5.902015


### Đánh giá và hậu xử lý

Mô hình không trả lại trực tiếp câu trả lời mà chỉ đưa ra xác xuất về vị trí bắt đầu và kết thúc của câu trả lời.

In [ ]:
import torch

if torch.cuda.is_available():
  torch.cuda.empty_cache()

model = AutoModelForQuestionAnswering.from_pretrained("csarron/bert-base-uncased-squad-v1")

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

Ta không cần quan tâm loss khi đánh giá chỉ cần giữ lại `'start_logits'` và `'end_logits'`

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([32, 384]), torch.Size([32, 384]))

Lấy ra kết quả tốt nhất

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 46,  57,  89,  43, 118,  44,  72,  42,  44,  41,  73,  41,  80,  45,
         156,  35,  40,  45,  80,  58,  77,  74,  42,  53,  41,  35,  42,  88,
          44,  44,  27, 133], device='cuda:0'),
 tensor([ 47,  58,  81,  44, 118, 109,  75,  11, 109,  42,  76,  42,  83,  45,
         159,  35,  83,  45,  83,  60,   0,  74,  43,  54,  42,  35,  43,  91,
          45,  45,  28, 133], device='cuda:0'))

Phương pháp chọn tốt nhất không khả thi????

Nên chọn cặp tốt nhất

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index:
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": ""
                }
            )

Viết hoàn chỉnh

In [ ]:
def prepare_validation_features(examples):

    # Tiền xử lý
    examples["question"] = [q.strip() for q in examples["question"]]

    # Tokenize
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # Để lưu lại vị trí feature thuộc vào example nào
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):

        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # Lưu lại id
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Đặt bằng None nếu như token không nằm trong context
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

Sử dụng `map` để chuyển tương tự như train

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Đưa ra dự đoán bằng phương thức predict

In [ ]:
raw_predictions = trainer.predict(validation_features)

`Trainer` giấu các cột không sử dụng bởi mô hình các chung ta cần để hậu xử lý nên ta cần đặt lại

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

Hậu xử lý

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]

context = datasets["validation"][0]["context"]

start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Loại bỏ những cặp nằm ngoài context
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Loại bỏ những cặp không tồn tại và độ dài lớn hơn max_answer_length
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index:
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 9.042664, 'text': 'Denver Broncos'},
 {'score': 7.7837133,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 6.7467585, 'text': 'Carolina Panthers'},
 {'score': 5.914267, 'text': 'Broncos'},
 {'score': 4.6553164,
  'text': 'Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 4.487817, 'text': 'Denver'},
 {'score': 3.6236343,
  'text': 'The American Football Conference (AFC) champion Denver Broncos'},
 {'score': 3.2325382,
  'text': 'American Football Conference (AFC) champion Denver Broncos'},
 {'score': 2.6190052,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10'},
 {'score': 2.5684674,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina'},
 {'score': 2.4616265, 'text': 'AFC) champion Denver Broncos'},
 {'score': 2.3646836,
  'text': 'The American Football Conference (A

So sánh với câu trả lời đúng

In [ ]:
datasets["validation"][0]["answers"]

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

Do là một dữ liệu có thể được chia làm nhiều feature nên ta cần phải tổng hợp lại và chọn ra câu trả lời tốt nhất.

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

Tạo ra hàm hậu xử lý

**Note:** cần sử dụng ngưỡng động (điểm của [CLS] để quyết định xem câu hỏi có câu trả lời hay không)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # Tạo từ điển để lưu đáp án
    predictions = collections.OrderedDict()

    # Log
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")


    for example_index, example in enumerate(tqdm(examples)):
        # Lấy tất cả feature ứng với dữ liệu
        feature_indices = features_per_example[example_index]

        min_null_score = None # Sử dụng là ngưỡng tối thiểu cho câu trả lời
        valid_answers = []

        context = example["context"]
        # Đi qua từng feature
        for feature_index in feature_indices:

            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            offset_mapping = features[feature_index]["offset_mapping"]

            # Cập nhật giá trị ngưỡng tối thiểu
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Đi qua tất cả các cặp
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:

                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue

                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}

        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

Áp dụng hàm vào dữ liệu được dự đoán ban đầu

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10784 features.


  0%|          | 0/10570 [00:00<?, ?it/s]

Tải metric để đánh giá

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

<ipython-input-73-69495f532713>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")


In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 60.65279091769158, 'f1': 69.87707803246407}

In [ ]:
# trainer.push_to_hub()

Chia sẻ mô hình `"username/modelname"`:

```python
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("my-name/my-EQA-model")
```

## Giải quyết bài toán theo hướng Sentence-Selection
---

Mục tiêu của bài toán này là tìm những câu trong văn bản liên quan tới câu hỏi. Notebook sẽ hướng dẫn finetune mô hình ngôn ngữ cho bài toán trả lời câu hỏi (extractive QA) theo hướng Sentence-Selection. Mặc dù phạm vi câu trả lời dự đoán bởi mô hình chưa thực sự sát như cách giải quyết theo hướng tìm vị trí (bắt đầu, kết thúc), tuy nhiên giải quyết theo cách này sẽ giảm độ phức tạp của dữ liệu cần xử lý. Đồng thời, bước này có thể được sử dụng để làm tiền đề cho các cách giải quyết phức tạp hơn. Bài toán có thể được mô tả như sau:

**Input:**
  - Question (câu hỏi)
  - Context (Đoạn văn bản cần tìm ra đoạn trả lời)

**Output**
  - Chỉ số các câu có nội dung liên quan tới câu hỏi.
---



In [ ]:
datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

### Tiền xử lý dữ liệu

Để giải quyết bài toán theo hướng Sentence-Selection, trước hết ta cần thực hiện một số bước sau:
- Chia context thành tập các câu.
- Sử dụng nhãn `answer_start` để xác định số thứ tự của câu chứa câu trả lời.
- Thực hiện việc ghép cặp (query-sentence)->label để tạo dữ liệu huấn luyện

In [ ]:
# Hàm thực hiện chia context thành các câu
import re
SPLIT_SENTENCE_REGEX = r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s"

def split_context_into_sentence(sample):
  list_sentence = re.split(SPLIT_SENTENCE_REGEX, sample.get('context'))
  return {
      **sample,
      "list_context_sentence": list_sentence
  }

_datasets = datasets.map(split_context_into_sentence)

# Với mục đích DEMO, chỉ sử dụng 100 sample
_datasets['train'] = _datasets['train'].select(range(100))
_datasets['validation'] = _datasets['validation'].select(range(100))

In [ ]:
# @title Bài tập 1: Viết hàm tìm ra số thứ tự của những sentence chứa answer

# Sentence chứa answer sẽ được gọi là label-sentence

# GỢI Ý:  Nên tìm vị trí của các label-sentence dựa vào các start_id của answer
def find_list_label_sentence(sample):
  list_label_sentence_id = []
  # YOUR CODE HERE
  return {
      **sample,
      "list_label_sentence_id": list_label_sentence_id
  }

_datasets_1 = _datasets.map(find_list_label_sentence)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
_datasets_1['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]},
 'list_context_sentence': ['Architecturally, the s

In [ ]:
# Hàm tạo cặp (query-sentence) -> label để thực hiện huấn luyện

def pair_creator(list_sample):
  batch_context_sentence = list_sample.get('list_context_sentence')
  batch_id = list_sample.get('id')
  batch_question = list_sample.get('question')
  batch_label_sentence_id = list_sample.get('list_label_sentence_id')

  batch_created_sample = {
      'id': [],
      'question': [],
      'label': [],
      'sentence': [],

  }
  for i, q in enumerate(batch_question):
    id = batch_id[i]
    list_context_sentence = batch_context_sentence[i]
    list_label_sentence_id = batch_label_sentence_id[i]
    for j, sentence in enumerate(list_context_sentence):
      sentence_label = int(j in list_label_sentence_id)
      batch_created_sample.get('id').append(id)
      batch_created_sample.get('question').append(q)
      batch_created_sample.get('label').append(sentence_label)
      batch_created_sample.get('sentence').append(sentence)

  return batch_created_sample

_datasets_2 = _datasets_1.map(pair_creator, batched=True, remove_columns=_datasets_1['train'].column_names)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
_datasets_2['train'][0]

{'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'label': 0,
 'sentence': 'Architecturally, the school has a Catholic character.'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Thực hiện tokenize

def tokenize_text_sentence_selection(examples):
  tokenized_examples = tokenizer(
          examples['question'],
          examples["sentence"],
          truncation="only_second",
          max_length=max_length,
          stride=doc_stride,
          return_overflowing_tokens=True,
          return_offsets_mapping=True,
          padding="max_length",
      )
  tokenized_examples['label'] = examples['label']
  # tokenized_examples['id'] = examples['id']
  return tokenized_examples

_dataset_3 = _datasets_2.map(tokenize_text_sentence_selection, batched=True, remove_columns=_datasets_2["train"].column_names)

Map:   0%|          | 0/716 [00:00<?, ? examples/s]

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

In [ ]:
_dataset_3['train'][0].keys()

dict_keys(['label', 'input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

### Fine-tuned mô hình BERTForSequenceClassificaiton

In [ ]:
from transformers import AutoModelForSequenceClassification

# Sử dụng model_checkpoint định nghĩa ở bước Tiền xử lý 1
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

In [ ]:
from transformers import TrainingArguments
from transformers import default_data_collator
from transformers import Trainer

# Định nghĩa TrainingArgument
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad-sentence-selection",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    # push_to_hub=True,
)

# Định nghĩa data_collator mặc định
data_collator = default_data_collator

# Định nghĩa đối tượng trainer cho mô hình
trainer = Trainer(
    model,
    args,
    train_dataset=_dataset_3["train"],
    eval_dataset=_dataset_3["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Huấn luyện và lưu mô hình
trainer.train()
trainer.save_model("test-squad-trained-sentence-selection")

# Lệnh để đẩy mô hình lên tài khoản cá nhân
# trainer.push_to_hub()

Epoch,Training Loss,Validation Loss
1,No log,0.665274


In [ ]:
trainer.evaluate()

{'eval_loss': 0.6652737259864807,
 'eval_runtime': 4.3471,
 'eval_samples_per_second': 81.663,
 'eval_steps_per_second': 5.291,
 'epoch': 1.0}

## Bài tập về nhà:
- Thực hiện các bước hậu xử lý và đánh giá hiệu quả của mô hình

**Tham khảo:** Code hậu xử lý và đánh giá ở phần trước.

Nộp notebook qua email: long.nh@vnu.edu.vn